In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import StopWordsRemover, Tokenizer, CountVectorizer, IDF
from pyspark.ml.clustering import KMeans
import nltk
from pyspark.sql import functions as F
from nltk.stem import WordNetLemmatizer
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.functions import to_date
from pyspark.sql.functions import when

In [2]:
# Définir une session Spark
spark = SparkSession.builder.appName("Traitement de dataset avec PySpark").getOrCreate()

# Fonction pour nettoyer les dates
def clean_date(date):
    if date is None:
        return date
    date = regexp_replace(date, '^121212', '')  # Supprimer '121212' au début de la date
    date = regexp_replace(date, '年|月|日', '-')  # Remplacer '年', '月', et '日' par '-'
    return date

# Charger le dataset
df = spark.read.csv("final_data_drone.csv", header=True, inferSchema=True)

# Appliquer la fonction de nettoyage aux colonnes concernées
cols_to_clean = ['priority date', 'filing/creation date', 'publication date', 'grant date']
for col_name in cols_to_clean:
    df = df.withColumn(col_name, clean_date(col(col_name)))
    df = df.withColumn(col_name, to_date(col(col_name)))
    df = df.withColumn(col_name, when(col(col_name).isNull(), '1900-01-01').otherwise(col(col_name)))
# Afficher le schéma du DataFrame
df.printSchema()

# Afficher les premières lignes pour vérification
df.show(200)

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- assignee: string (nullable = true)
 |-- author : string (nullable = true)
 |-- priority date: string (nullable = true)
 |-- filing/creation date: string (nullable = true)
 |-- publication date: string (nullable = true)
 |-- grant date: string (nullable = true)
 |-- result_link: string (nullable = true)
 |-- figure_link: string (nullable = true)

+------------------+--------------------+-------------------------------------+-----------------------------------+-------------+--------------------+----------------+----------+--------------------+--------------------+
|                id|               title|                             assignee|                            author |priority date|filing/creation date|publication date|grant date|         result_link|         figure_link|
+------------------+--------------------+-------------------------------------+-----------------------------------+-------------+

In [3]:
from pyspark.sql.functions import regexp_replace

# Supprimer '-' à la fin des valeurs
df = df.withColumn("priority date", regexp_replace("priority date", "-$", ""))
df = df.withColumn("filing/creation date", regexp_replace("filing/creation date", "-$", ""))
df = df.withColumn("publication date", regexp_replace("publication date", "-$", ""))
df = df.withColumn("grant date", regexp_replace("grant date", "-$", ""))

# Afficher le DataFrame après nettoyage
df.show()


+----------------+--------------------+------------------------------+--------------------------------+-------------+--------------------+----------------+----------+--------------------+--------------------+
|              id|               title|                      assignee|                         author |priority date|filing/creation date|publication date|grant date|         result_link|         figure_link|
+----------------+--------------------+------------------------------+--------------------------------+-------------+--------------------+----------------+----------+--------------------+--------------------+
|  US-10440323-B2|Facilitating wide...|                   Wellen Sham|                     Wellen Sham|   2015-12-31|          2018-03-20|      2019-10-08|2019-10-08|https://patents.g...|https://patentima...|
|US-2023341849-A1|Deployment mechan...|                Skyfront Corp.|                    Troy Mestler|   2020-04-15|          2021-09-30|      2023-10-26|1900-01-0

In [4]:
df = df.dropDuplicates(["ID"])


In [5]:
# Nombre de lignes dans le DataFrame
num_rows = df.count()
print("Nombre de lignes dans le DataFrame :", num_rows)
df.printSchema()


Nombre de lignes dans le DataFrame : 1970
root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- assignee: string (nullable = true)
 |-- author : string (nullable = true)
 |-- priority date: string (nullable = true)
 |-- filing/creation date: string (nullable = true)
 |-- publication date: string (nullable = true)
 |-- grant date: string (nullable = true)
 |-- result_link: string (nullable = true)
 |-- figure_link: string (nullable = true)



# **Cubes**

# **Cube 1D**

In [6]:
from pyspark.sql.functions import desc

# Cube pour les top 10 mots-clés
top_10_keywords = df.groupBy("title").count().orderBy(desc("count")).limit(10)

# Cube pour les top 20 mots-clés
top_20_keywords = df.groupBy("title").count().orderBy(desc("count")).limit(20)

# Cube pour les inventeurs/auteurs
# Cube pour les inventeurs/auteurs
inventors_authors = df.groupBy("author ").count().orderBy(desc("count"))

# Affichage des résultats
print("Top 10 mots-clés:")
top_10_keywords.show()

print("Top 20 mots-clés:")
top_20_keywords.show()

print("Auteurs:")
inventors_authors.show()


Top 10 mots-clés:
+--------------------+-----+
|               title|count|
+--------------------+-----+
|              Drone |   66|
|A kind of plant p...|    4|
|     Drone aircraft |    3|
|       Drone system |    3|
|Quadcopter model ...|    3|
|         Drone port |    3|
|      Drone Station |    2|
|Autonomous perfor...|    2|
|Transformable drone |    2|
|Methods and syste...|    2|
+--------------------+-----+

Top 20 mots-clés:
+--------------------+-----+
|               title|count|
+--------------------+-----+
|              Drone |   66|
|A kind of plant p...|    4|
|       Drone system |    3|
|     Drone aircraft |    3|
|Quadcopter model ...|    3|
|         Drone port |    3|
|      Drone Station |    2|
|Motor for drone a...|    2|
|Autonomous perfor...|    2|
|Transformable drone |    2|
|Methods and syste...|    2|
|Portable folding ...|    2|
|System for monito...|    2|
|Drone charging st...|    2|
|Secure recovery s...|    2|
|Defense drone to ...|    2|
|Syste

# **Cube 2D**

In [7]:
# Cube 2D pour l'analyse des assignees et des inventeurs/auteurs
cube_assignee_author = df.groupBy("assignee", "author ").count().orderBy(desc("count"))

# Cube 2D pour l'analyse des assignees et des mots-clés
cube_assignee_title = df.groupBy("assignee", "title").count().orderBy(desc("count"))

# Cube 2D pour l'analyse des assignees et des dates de priorité
cube_assignee_priority_date = df.groupBy("assignee", "priority date").count().orderBy(desc("count"))

# Affichage des résultats
print("Cube 2D Assignee vs. Author:")
cube_assignee_author.show()



Cube 2D Assignee vs. Author:
+--------------------------+------------------------------+-----+
|                  assignee|                       author |count|
+--------------------------+------------------------------+-----+
|         ETAK Systems, LLC|                    Lee Priest|    6|
|      Samsung Electroni...|                    Minsoo Kim|    5|
|         Metal Raptor, Llc|                    Lee Priest|    5|
|    株式会社ナイルワークス|千大 和氣, 千大 和氣, 洋 柳...|    4|
|      Guangdong Shiji T...|                      Jun Zhou|    4|
|      International Bus...|          Michael S. Gordon...|    4|
|             주식회사 숨비|                        오인선|    4|
|    清远市巨劲科技有限公司|                        杨毅勋|    4|
|           주식회사 휴인스|                        송태훈|    4|
|    株式会社ナイルワークス|            千大 和氣, 洋 柳下|    4|
|      Shanghai Autoflig...|                    Haofeng Tu|    4|
|广东莱盛隆电子股份有限公司|                        罗文星|    4|
|      Shenzhen Hubsan T...|                   Guangjun Li|    4|
|  

In [8]:
print("Cube 2D Assignee vs. Title:")
cube_assignee_title.show()

Cube 2D Assignee vs. Title:
+-------------------------------+--------------------+-----+
|                       assignee|               title|count|
+-------------------------------+--------------------+-----+
|           Samsung Electroni...|              Drone |   11|
|           Shenzhen Hubsan T...|Quadcopter model ...|    3|
|           Guangdong Shiji T...|              Drone |    3|
|Ｉｈｉ運搬機械株式会社, Ihi ...|         Drone port |    2|
|                 DuckDrone, LLC|Drone-target hunt...|    2|
|            엘지이노텍 주식회사|Motor for drone a...|    2|
|             Neu Robotics, Inc.|              Drone |    2|
|            Walmart Apollo, Llc|Systems and metho...|    2|
|         上海拓攻机器人有限公司|Plant protection ...|    2|
|               Michael Gavrilov|Drone systems for...|    2|
|株式会社ナイルワークス, Nile...|       Drone system |    2|
|           ZEROTECH (Shenzhe...|              Drone |    2|
|       寧波派麗肯無人機有限公司|              Drone |    2|
|           Guangdong Attop T...|              Dro

In [9]:

print("Cube 2D Assignee vs. Priority Date:")
cube_assignee_priority_date.show()

Cube 2D Assignee vs. Priority Date:
+-------------------------------------+-------------+-----+
|                             assignee|priority date|count|
+-------------------------------------+-------------+-----+
|                    ETAK Systems, LLC|   2016-06-10|    7|
|                 Samsung Electroni...|   2016-10-18|    6|
|               清远市巨劲科技有限公司|   2018-03-15|    5|
|                    Metal Raptor, Llc|   2016-06-10|    5|
|               株式会社ナイルワークス|   2018-02-28|    4|
|                 DynaEnergetics Eu...|   2018-05-31|    4|
|                 International Bus...|   2016-03-08|    4|
|                 Shanghai Autoflig...|   2017-05-05|    4|
|                 Guangdong Shiji T...|   2018-08-10|    3|
|                 Samsung Electroni...|   2016-04-19|    3|
|                    Alshdaifat, Wasfi|   2014-03-25|    3|
|                공간정보기술 주식회사|   2017-11-07|    3|
|                             (주)지트|   2017-05-31|    3|
|             寧波派麗肯無人機有限公司|   2017-06-2

# **Cube 3D**

In [ ]:
# Cube 3D pour l'analyse des assignees, des inventeurs/auteurs et des titres
cube_assignee_author_title = df.groupBy("assignee", "inventor/author ", "title").count().orderBy(desc("count"))
# Affichage des résultats
print("Cube 3D Assignee vs. Author vs. Title:")
cube_assignee_author_title.show()

Cube 3D Assignee vs. Author vs. Title:
+------------------------+--------------------------------+--------------------+-----+
|                assignee|                inventor/author |               title|count|
+------------------------+--------------------------------+--------------------+-----+
|    Samsung Electroni...|                      Minsoo Kim|              Drone |    5|
|    Guangdong Shiji T...|                        Jun Zhou|              Drone |    3|
|    Michael Steward E...|            Michael Steward E...|Drone charging st...|    2|
|    Samsung Electroni...|            Eunyoung Kim, Hay...|              Drone |    2|
|    ZEROTECH (Shenzhe...|                      Wenfeng Li|              Drone |    2|
|        Michael Gavrilov|                Michael Gavrilov|Drone systems for...|    2|
|     엘지이노텍 주식회사|                          박영대|Motor for drone a...|    2|
|      Neu Robotics, Inc.|            Cyril Lutterodt, ...|              Drone |    2|
|    Samsung Ele

In [ ]:
# Cube 3D pour l'analyse des assignees, des dates de priorité et des titres
cube_assignee_priority_title = df.groupBy("assignee", "priority date", "title").count().orderBy(desc("count"))
print("Cube 3D Assignee vs. Priority Date vs. Title:")
cube_assignee_priority_title.show()

Cube 3D Assignee vs. Priority Date vs. Title:
+-----------------------------------+-------------+--------------------+-----+
|                           assignee|priority date|               title|count|
+-----------------------------------+-------------+--------------------+-----+
|               Samsung Electroni...|   2016-10-18|              Drone |    6|
|               Samsung Electroni...|   2016-04-19|              Drone |    3|
|                   Michael Gavrilov|   2019-06-29|Drone systems for...|    2|
|                 Neu Robotics, Inc.|   2017-08-14|              Drone |    2|
|           寧波派麗肯無人機有限公司|   2017-06-21|              Drone |    2|
|               Samsung Electroni...|   2016-08-05|              Drone |    2|
|                     DuckDrone, LLC|   2016-01-07|Drone-target hunt...|    2|
|               Guangdong Shiji T...|   2018-08-10|              Drone |    2|
|                   Spin Master Ltd.|   2018-02-16|              Drone |    2|
|               Sh

In [ ]:
# Cube 3D pour l'analyse des assignees, des inventeurs/auteurs et des dates de priorité
cube_assignee_author_priority = df.groupBy("assignee", "inventor/author ", "priority date").count().orderBy(desc("count"))
print("Cube 3D Assignee vs. Author vs. Priority Date:")
cube_assignee_author_priority.show()

Cube 3D Assignee vs. Author vs. Priority Date:
+------------------------+-------------------------------+-------------+-----+
|                assignee|               inventor/author |priority date|count|
+------------------------+-------------------------------+-------------+-----+
|       ETAK Systems, LLC|                     Lee Priest|   2016-06-10|    5|
|       Metal Raptor, Llc|                     Lee Priest|   2016-06-10|    5|
|    Samsung Electroni...|                     Minsoo Kim|   2016-10-18|    5|
|    Shanghai Autoflig...|                     Haofeng Tu|   2017-05-05|    4|
|  清远市巨劲科技有限公司|                         杨毅勋|   2018-03-15|    4|
|       Alshdaifat, Wasfi|           Eida Almuhairbi, ...|   2014-03-25|    3|
|寧波派麗肯無人機有限公司|                   滔 干, 滔 干|   2017-06-21|    3|
|                (주)지트|         이준섭, 조지현, 최영재|   2017-05-31|    3|
|    Guangdong Shiji T...|                       Jun Zhou|   2018-08-10|    3|
|          다인시스템(주)|         안승열, 황선우, 강순도|   2

In [ ]:
# Cube 3D pour l'analyse des assignees, des inventeurs/auteurs et des dates de création
cube_assignee_author_creation = df.groupBy("assignee", "inventor/author ", "filing/creation date").count().orderBy(desc("count"))
print("Cube 3D Assignee vs. Author vs. Creation Date:")
cube_assignee_author_creation.show()

Cube 3D Assignee vs. Author vs. Creation Date:
+------------------------+-------------------------------+--------------------+-----+
|                assignee|               inventor/author |filing/creation date|count|
+------------------------+-------------------------------+--------------------+-----+
|    Samsung Electroni...|                     Minsoo Kim|          2017-04-17|    5|
|  清远市巨劲科技有限公司|                         杨毅勋|          2018-03-15|    4|
|    Shanghai Autoflig...|                     Haofeng Tu|          2018-05-04|    4|
|       Alshdaifat, Wasfi|           Eida Almuhairbi, ...|          2014-03-25|    3|
|                (주)지트|         이준섭, 조지현, 최영재|          2017-05-31|    3|
|寧波派麗肯無人機有限公司|                   滔 干, 滔 干|          2017-07-14|    3|
|    Guangdong Shiji T...|                       Jun Zhou|          2018-08-10|    3|
|       주식회사 에어퓨쳐|                         유윤종|          2021-03-15|    2|
|       Metal Raptor, Llc|                     Lee Priest|  

In [ ]:
# Cube 3D pour l'analyse des assignees, des dates de priorité et des dates de création
cube_assignee_priority_creation = df.groupBy("assignee", "priority date", "filing/creation date").count().orderBy(desc("count"))
print("Cube 3D Assignee vs. Priority Date vs. Creation Date:")
cube_assignee_priority_creation.show()

Cube 3D Assignee vs. Priority Date vs. Creation Date:
+-------------------------------------+-------------+--------------------+-----+
|                             assignee|priority date|filing/creation date|count|
+-------------------------------------+-------------+--------------------+-----+
|                 Samsung Electroni...|   2016-10-18|          2017-04-17|    6|
|               清远市巨劲科技有限公司|   2018-03-15|          2018-03-15|    5|
|                 Shanghai Autoflig...|   2017-05-05|          2018-05-04|    4|
|             寧波派麗肯無人機有限公司|   2017-06-21|          2017-07-14|    3|
|               株式会社ナイルワークス|   2018-02-28|          2019-02-27|    3|
|                    Alshdaifat, Wasfi|   2014-03-25|          2014-03-25|    3|
|                 Guangdong Shiji T...|   2018-08-10|          2018-08-10|    3|
|                 Samsung Electroni...|   2016-04-19|          2016-10-18|    3|
|                공간정보기술 주식회사|   2017-11-07|          2017-11-07|    3|
|                 

In [ ]:
# Cube 3D pour l'analyse des assignees, des inventeurs/auteurs et des dates de publication
cube_assignee_author_publication = df.groupBy("assignee", "inventor/author ", "publication date").count().orderBy(desc("count"))
print("Cube 3D Assignee vs. Author vs. Publication Date:")
cube_assignee_author_publication.show()

Cube 3D Assignee vs. Author vs. Publication Date:
+--------------------------------------+-------------------------------+----------------+-----+
|                              assignee|               inventor/author |publication date|count|
+--------------------------------------+-------------------------------+----------------+-----+
|                  Samsung Electroni...|                     Minsoo Kim|      2019-04-23|    5|
|                清远市巨劲科技有限公司|                         杨毅勋|      2018-10-19|    4|
|                     Alshdaifat, Wasfi|           Eida Almuhairbi, ...|      2014-05-30|    3|
|                              (주)지트|         이준섭, 조지현, 최영재|      2017-09-26|    2|
|                       주식회사 휴인스|                         송태훈|      2018-12-28|    2|
|                      MerchSource, LLC|               Crispian Tompkin|      2018-08-14|    2|
|         전주대학교 산학협력단, 김동현|                         김동현|      2020-06-18|    2|
|     株式会社ゼンリンデータコム, Ze...|       辰彦 清水, T

In [ ]:
# Exporter le DataFrame traité vers un fichier CSV
df.write.csv("DATA_for_viz.csv", header=True, mode="overwrite")

In [ ]:
# prompt: comment exporter tous les cubes vers execel chacun dans une feuille
import pandas as pd
!pip install xlsxwriter
# Create a Pandas DataFrame for each cube
cube_assignee_author_pd = cube_assignee_author.toPandas()
cube_assignee_title_pd = cube_assignee_title.toPandas()
cube_assignee_priority_date_pd = cube_assignee_priority_date.toPandas()
cube_assignee_author_title_pd = cube_assignee_author_title.toPandas()
cube_assignee_priority_title_pd = cube_assignee_priority_title.toPandas()
cube_assignee_author_priority_pd = cube_assignee_author_priority.toPandas()
cube_assignee_author_creation_pd = cube_assignee_author_creation.toPandas()
cube_assignee_priority_creation_pd = cube_assignee_priority_creation.toPandas()
cube_assignee_author_publication_pd = cube_assignee_author_publication.toPandas()

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('cubes.xlsx', engine='xlsxwriter')

# Change the sheet names to be within the 31 character limit
cube_assignee_author_pd.to_excel(writer, sheet_name='Cube Assignee_Inventor')
cube_assignee_title_pd.to_excel(writer, sheet_name='Cube Assignee_Title')
cube_assignee_priority_date_pd.to_excel(writer, sheet_name='Cube Assignee_PriorityDate')
cube_assignee_author_title_pd.to_excel(writer, sheet_name='Cube Assignee_Author_Title')
cube_assignee_priority_title_pd.to_excel(writer, sheet_name='Cube Assignee_Priority_Title')
cube_assignee_author_priority_pd.to_excel(writer, sheet_name='Cube Assignee_Author_Priority')
cube_assignee_author_creation_pd.to_excel(writer, sheet_name='Cube Assignee_Author_Creation')
cube_assignee_priority_creation_pd.to_excel(writer, sheet_name='Cube Assignee_Priority_Creation')
cube_assignee_author_publication_pd.to_excel(writer, sheet_name='Cube Assignee_Author_Public')

writer.close()

